In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
conn = sqlite3.connect('chembl_23\\chembl_23_sqlite\\chembl_23.db')

In [3]:
c = conn.cursor()

## Table in the SQLite database

In [4]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
for item in c.fetchall():
    print(item)

('version',)
('chembl_id_lookup',)
('source',)
('docs',)
('research_stem',)
('structural_alert_sets',)
('bio_component_sequences',)
('molecule_dictionary',)
('research_companies',)
('structural_alerts',)
('compound_properties',)
('compound_records',)
('molecule_hierarchy',)
('molecule_synonyms',)
('biotherapeutics',)
('compound_structural_alerts',)
('compound_structures',)
('biotherapeutic_components',)
('target_type',)
('organism_class',)
('protein_family_classification',)
('protein_classification',)
('go_classification',)
('component_sequences',)
('variant_sequences',)
('target_dictionary',)
('component_class',)
('component_synonyms',)
('cell_dictionary',)
('protein_class_synonyms',)
('tissue_dictionary',)
('component_go',)
('target_components',)
('target_relations',)
('domains',)
('component_domains',)
('binding_sites',)
('site_components',)
('assay_type',)
('relationship_type',)
('confidence_score_lookup',)
('curation_lookup',)
('activity_stds_lookup',)
('data_validity_lookup',)
('

## Columns in particular table

In [9]:
table_name = 'ACTIVITIES'
c.execute('PRAGMA TABLE_INFO({})'.format(table_name))

# collect names in a list
names = [tup[1] for tup in c.fetchall()]
print(names)

['activity_id', 'assay_id', 'doc_id', 'record_id', 'molregno', 'standard_relation', 'published_value', 'published_units', 'standard_value', 'standard_units', 'standard_flag', 'standard_type', 'activity_comment', 'published_type', 'data_validity_comment', 'potential_duplicate', 'published_relation', 'pchembl_value', 'bao_endpoint', 'uo_units', 'qudt_units']


## Count the structures as a function of SMILES code length

In [6]:
c.execute('select count(*) from (select molregno, canonical_smiles, length(canonical_smiles) from COMPOUND_STRUCTURES \
           where length(canonical_smiles) > 210)')
c.fetchmany(10)

[(18461,)]

## Chosen query (takes all activities of type GI50 and IC50 where the activity values are not NULL) 

In [10]:
columns = ['molregno', 'canonical_smiles', 'activity_id', 'published_value', 
           'published_units', 'standard_value', 'standard_units', 'standard_type', 'activity_comment']
c.execute("select CS.molregno, \
           CS.canonical_smiles, \
           AC.activity_id, \
           AC.published_value, \
           AC.published_units, \
           AC.standard_value, \
           AC.standard_units, \
           AC.standard_type, \
           AC.activity_comment \
           from COMPOUND_STRUCTURES CS \
           inner join ACTIVITIES AC on CS.molregno = AC.molregno \
           and (AC.standard_type = 'IC50' or AC.standard_type = 'GI50') \
           and (AC.standard_value IS NOT NULL or AC.published_value IS NOT NULL)")
#and AC.standard_units = 'M')")
#where CS.canonical_smiles = 'Cn1c(\C=C\c2ccc(cc2C(=O)O)C#N)cc3ccccc13' \
#where AC.standard_type = 'IC50' and AC.published_units = 'uM'")
#where AC.published_units = 'nM' and length(CS.canonical_smiles) > 210 \
#pd.DataFrame(c.fetchmany(1000), columns = columns)
DF_structures = pd.DataFrame(c.fetchall(), columns = columns)

In [11]:
DF_structures

,molregno,canonical_smiles,activity_id,published_value,published_units,standard_value,standard_units,standard_type,activity_comment
0,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35160,1.750000e+01,uM,1.750000e+04,nM,GI50,None
1,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35161,4.040000e+01,uM,4.040000e+04,nM,GI50,None
2,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35163,3.330000e+01,uM,3.330000e+04,nM,GI50,None
3,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35164,4.230000e+01,uM,4.230000e+04,nM,GI50,None
4,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35165,1.000000e+02,uM,1.000000e+05,nM,GI50,None
5,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35166,3.590000e+01,uM,3.590000e+04,nM,GI50,None
6,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35167,1.000000e+02,uM,1.000000e+05,nM,GI50,None
7,425503,C[C@@H]1[C@]2(O[C@H]3C=C4[C@@H]5CC[C@H]6Cc7nc8...,35783,1.000000e+03,nM,1.000000e+03,nM,GI50,None
8,183969,COc1cc2C(=O)N(C)C3=C(\C(=C/CCCI)\c4cc5OCOc5cc3...,36363,2.760000e+01,uM,2.760000e+04,nM,GI50,None
9,183969,COc1cc2C(=O)N(C)C3=C(\C(=C/CCCI)\c4cc5OCOc5cc3...,36364,5.600000e-01,uM,5.600000e+02,nM,GI50,None


### Store on file

In [12]:
DF_structures.to_csv('DF_GI50_IC50_all.csv', index = False)

In [40]:
DF_structures.head(50)

,molregno,canonical_smiles,activity_id,published_value,published_units,standard_value,standard_units,standard_type,activity_comment
0,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35160,1.750000e+01,uM,1.750000e+04,nM,GI50,None
1,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35161,4.040000e+01,uM,4.040000e+04,nM,GI50,None
2,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35163,3.330000e+01,uM,3.330000e+04,nM,GI50,None
3,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35164,4.230000e+01,uM,4.230000e+04,nM,GI50,None
4,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35165,1.000000e+02,uM,1.000000e+05,nM,GI50,None
5,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35166,3.590000e+01,uM,3.590000e+04,nM,GI50,None
6,184040,COc1cc2C(=O)N(C)C3=C(\C(=C\CCCCl)\c4cc5OCOc5cc...,35167,1.000000e+02,uM,1.000000e+05,nM,GI50,None
7,425503,C[C@@H]1[C@]2(O[C@H]3C=C4[C@@H]5CC[C@H]6Cc7nc8...,35783,1.000000e+03,nM,1.000000e+03,nM,GI50,None
8,183969,COc1cc2C(=O)N(C)C3=C(\C(=C/CCCI)\c4cc5OCOc5cc3...,36363,2.760000e+01,uM,2.760000e+04,nM,GI50,None
9,183969,COc1cc2C(=O)N(C)C3=C(\C(=C/CCCI)\c4cc5OCOc5cc3...,36364,5.600000e-01,uM,5.600000e+02,nM,GI50,None


In [63]:
np.sum(~DF_structures.loc[:, 'standard_units'].isna())

4121503

In [61]:
~DF_structures.loc[:, 'standard_units'].isna().values

array([ True,  True,  True, ...,  True,  True,  True])

In [65]:
DF_structures = DF_structures[~DF_structures.loc[:, 'standard_units'].isna()]

### Groupby definition + aggregation functions

In [66]:
grouped = DF_structures.groupby('molregno')

In [67]:
aggFunctions = {
                'molregno': ['min', 'count'], 'canonical_smiles': 'min',
                'published_value': ['min', 'max', 'mean'],
                #'published_units': 'min',
                'standard_value': ['min', 'max', 'mean'],
                'standard_units': 'max',
                'standard_type': 'min',
                #'activity_comment': 'min'
                }

In [68]:
DF_structuresGrouped = grouped.agg(aggFunctions)

In [69]:
DF_structuresGrouped.tail(5)

molregno                                         canonical_smiles  \
              min count                                                min   
molregno                                                                     
2096031   2096031     1  CC(CC(=O)C[C@](C)(O)[C@H]1CC(=O)[C@@]2(C)C3=C(...   
2096032   2096032     2  C(Oc1ccc2[nH]nc(c3nc4cc(ccc4[nH]3)N5CCCCC5)c2c...   
2096033   2096033     2  Clc1cnccc1COc2ccc3[nH]nc(c4nc5cc(ccc5[nH]4)N6C...   
2096034   2096034     2  Clc1cncc(Cl)c1COc2ccc3[nH]nc(c4nc5cc(ccc5[nH]4...   
2096152   2096152     7  COC(=O)CCc1c(C)c2cc3nc(cc4[nH]c(cc5nc(cc1[nH]2...   

         published_value                       standard_value           \
                     min        max       mean            min      max   
molregno                                                                 
2096031          21.3300    21.3300    21.3300        21330.0  21330.0   
2096032          20.8000  5016.9000  2518.8500           20.8   5016.9   
2096033          13.6000  3071.1000  1542.3500           13.6   3071.1   
2096034           6.6000   158.7000    82.6500            6.6    158.7   
2096152           3.5296     9.6205     5.6461         3529.6   9620.5   

                   standard_units standard_type  
              mean            max           min  
molregno                                         
2096031   21330.00             nM          IC50  
2096032    2518.85             nM          IC50  
2096033    1542.35             nM          IC50  
2096034      82.65             nM          IC50  
2096152    5646.10             nM          IC50

In [70]:
DF_structuresGrouped.to_csv('DF_GI50_IC50_grouped_20180612.csv', index = False)

Same min and max, homogenous group of rows of active species 

In [83]:
sameActivities = DF_structuresGrouped.loc[:, 'standard_value'].loc[:, 'min'] == DF_structuresGrouped.loc[:, 'standard_value'].loc[:, 'max']

In [85]:
DF_structuresGrouped = DF_structuresGrouped[sameActivities]

In [86]:
DF_structuresGrouped.to_csv('DF_GI50_IC50_grouped_20180612.csv', index = False)

## Full query (multiple joins)

In [ ]:
columns = ['molregno', 'canonical_smiles', 'alogp', 'aromatic_rings', 'full_mwt', 'activity_id', 'published_value', 
           'published_units', 'standard_value', 'standard_units', 'standard_type', 'DOC: title', 'ASSY: description']
c.execute("select min(canonical_smiles), min(standard_units), max(standard_value) from (select CS.molregno, \
           CS.canonical_smiles, \
           CP.alogp, \
           CP.aromatic_rings, \
           CP.full_mwt, \
           AC.activity_id, \
           AC.published_value, \
           AC.published_units, \
           AC.standard_value, \
           AC.standard_units, \
           AC.standard_type, \
           DO.title, \
           ASSY.description \
           from COMPOUND_STRUCTURES CS \
           inner join COMPOUND_PROPERTIES CP on CS.molregno = CP.molregno \
           inner join ACTIVITIES AC on CS.molregno = AC.molregno \
           inner join DOCS DO on AC.doc_id = DO.doc_id \
           inner join ASSAYS ASSY on AC.assay_id = ASSY.assay_id \
           where AC.published_units = 'uM'\
           group by CS.molregno)")
#where AC.standard_type = 'IC50' and AC.published_units = 'uM'")
pd.DataFrame(c.fetchmany(100))
#DF_IC50 = pd.DataFrame(c.fetchmany(1000), columns = columns)

In [5]:
DF_IC50['standard_value'] = DF_IC50['standard_value'].astype('float')
DF_IC50.loc[:, 'negLogIC'] = pd.Series( -1.0 * np.log10(DF_IC50['standard_value']), index = DF_IC50.index )

## Get the units distribution of the ACTIVITY

In [36]:
c.execute("select min(standard_units), count(*) DESCENDING from ACTIVITIES group by standard_units")
#where AC.standard_type = 'IC50' and AC.published_units = 'uM'")
pd.DataFrame(c.fetchall(), columns = ['unit', 'count']).sort_values('count', ascending = False)

,unit,count
1403,nM,10051691
0,None,1612336
1,%,1512432
1992,ug.mL-1,673717
1830,uM,98469
1340,mm,78990
977,hr,58959
1973,ug ml-1,58864
1181,mg.kg-1,57280
1144,mg kg-1,27610


## Get the activity type distribution

In [37]:
c.execute("select min(standard_type), count(*) DESCENDING from ACTIVITIES group by standard_type")
#where AC.standard_type = 'IC50' and AC.published_units = 'uM'")
pd.DataFrame(c.fetchall(), columns = ['type', 'count']).sort_values('count', ascending = False)

,type,count
4279,Potency,4598151
2057,GI50,2569096
2349,IC50,1820108
2520,Inhibition,836240
316,Activity,768774
2777,Ki,648840
3330,MIC,512481
1674,EC50,358198
148,AC50,156778
1734,ED50,98633


## Column names

In [55]:
table_name = 'ASSAYS'
c.execute('PRAGMA TABLE_INFO({})'.format(table_name))

# collect names in a list
names = [tup[1] for tup in c.fetchall()]
print(names)

['assay_id', 'doc_id', 'description', 'assay_type', 'assay_test_type', 'assay_category', 'assay_organism', 'assay_tax_id', 'assay_strain', 'assay_tissue', 'assay_cell_type', 'assay_subcellular_fraction', 'tid', 'relationship_type', 'confidence_score', 'curated_by', 'src_id', 'src_assay_id', 'chembl_id', 'cell_id', 'bao_format', 'tissue_id', 'variant_id']


In [71]:
table_name = 'ACTIVITIES'
c.execute('PRAGMA TABLE_INFO({})'.format(table_name))

# collect names in a list
names = [tup[1] for tup in c.fetchall()]
print(names)

['activity_id', 'assay_id', 'doc_id', 'record_id', 'molregno', 'standard_relation', 'published_value', 'published_units', 'standard_value', 'standard_units', 'standard_flag', 'standard_type', 'activity_comment', 'published_type', 'data_validity_comment', 'potential_duplicate', 'published_relation', 'pchembl_value', 'bao_endpoint', 'uo_units', 'qudt_units']


In [24]:
conn.close()